In [132]:
## Multi- Spectrum Fitting by Segment

import numpy as np
import numpy.ma as ma
import pandas as pd
import qgrid
import os
from lmfit import minimize, Parameters, report_fit, Model, Minimizer
from lmfit.models import GaussianModel, LinearModel, Model, VoigtModel
import matplotlib.pyplot as plt
from matplotlib import gridspec

import sys
sys.path.append(r'C:\Users\ema3\Documents\Python Scripts\HAPI')# set location of HAPI.py module
sys.path.append(r'C:\Users\ema3\Documents\Cold Cavity - O2 A Band')# set location of HAPI.py module
%matplotlib inline
from hapi import *
from hapi_HTP_definition_dataframe_190222 import *

import seaborn as sns
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.set_context("poster")


In [133]:
path = r'C:\Users\ema3\Documents\Data\New Oxygen' # Location of the Summary Data File
os.chdir(path)

#Generic Fit Parameters
wave_range = 1.5 #range outside of experimental x-range to simulate
IntensityThreshold = 1e-30 #intensities must be above this value to be simulated
Fit_Intensity = 1e-25 #intensities must be above this value for the line to be fit
order_baseline_fit = 2


#Define all Spectra individually
spectrum_80 = Spectrum('15Feb19_80Torr Air_ oxygen Computer _new config_corr', 
                       concentration = { 7 : 0.2072}, natural_abundance = True, diluent = 'air', 
                       etalons = {1: [0.001455, 0.83629422058], 2: [ 6.2325e-04, 1.67258844116]})
spectrum_100 = Spectrum('15Feb19_100Torr Air_ oxygen Computer _new config_corr', 
                        concentration = { 7 : 0.2072}, natural_abundance = True, diluent = 'air', 
                        etalons = {1: [0.001455, 0.83629422058], 2: [ 6.2325e-04, 1.67258844116]})
spectrum_120 = Spectrum('15Feb19_120Torr Air_ Oxygen Computer _new config_corr', 
                        concentration = { 7 : 0.2072}, natural_abundance = True, diluent = 'air',
                        etalons = {1: [0.001455, 0.83629422058], 2: [ 6.2325e-04, 1.67258844116]})

#Add all spectrum to a Dataset object
SPECTRA = Dataset([spectrum_80, spectrum_100, spectrum_120], 'New Configuration Oxygen A-Band', baseline_order = order_baseline_fit)

#Generate Baseline Parameter list based on number of etalons in spectra definitions and baseline order
BASE_LINELIST = SPECTRA.generate_baseline_paramlist()

#Read in Possible linelists
hapi = r'C:\Users\ema3\Documents\Python Scripts\HAPI'
os.chdir(hapi)
PARAM_LINELIST = pd.read_csv('Drouin_2017_linelist.csv')


In [162]:
os.chdir(path)
# Set-up for Fitting
lineprofile = 'SDVP' #VP, SDVP, NGP, SDNGP, HTP


class Generate_FitParam_File:
    def __init__ (self, dataset, param_linelist, base_linelist, 
                  lineprofile = 'VP', linemixing = False, threshold_intensity = 1e-30, fit_intensity = 1e-25, fit_window = 1.5, sim_window = 5, 
                  param_linelist_savename = 'Parameter_LineList', base_linelist_savename = 'Baseline_LineList', 
                 nu_constrain = True, sw_constrain = False, gamma0_constrain = True, delta0_constrain = True, aw_constrain = True, as_constrain = True, 
                 nuVC_constrain = True, eta_constrain =True, linemixing_constrain = True):
        self.dataset = dataset
        self.param_linelist = param_linelist
        self.base_linelist = base_linelist
        self.lineprofile = lineprofile
        self.linemixing = linemixing
        self.threshold_intensity = threshold_intensity
        self.fit_intensity = fit_intensity
        self.fit_window = fit_window
        self.sim_window = sim_window
        self.base_linelist_savename = base_linelist_savename
        self.param_linelist_savename = param_linelist_savename
        self.nu_constrain = nu_constrain
        self.sw_constrain = sw_constrain
        self.gamma0_constrain = gamma0_constrain
        self.delta0_constrain = delta0_constrain
        self.aw_constrain = aw_constrain
        self.as_constrain = as_constrain
        self.nuVC_constrain = nuVC_constrain
        self.eta_constrain = eta_constrain
        self.linemixing_constrain = linemixing_constrain
    # Do I need getters and setters or will this just be re-run if needed
    def get_dataset(self):
        return self.dataset
    def get_param_linelist(self):
        return self.param_linelist
    def get_base_linelist(self):
        return self.base_linelist
    def generate_fit_param_linelist_from_linelist(self, vary_nu = {7:{1:True, 2:False, 3:False}, 1:{1:False}}, vary_sw = {7:{1:True, 2:False, 3:False}},vary_gamma0 = {7:{1: True, 2:False, 3: False}, 1:{1:False}}, vary_n_gamma0 = {}, 
                                   vary_delta0 = {7:{1: True, 2:False, 3: False}, 1:{1:False}}, vary_n_delta0 = {}, 
                                   vary_aw = {7:{1: True, 2:False, 3: False}, 1:{1:False}}, vary_n_gamma2 = {}, 
                                   vary_as = {}, vary_n_delta2 = {}, 
                                   vary_nuVC = {}, vary_n_nuVC = {},
                                   vary_eta = {}, vary_linemixing = {}):
        param_linelist_df = self.get_param_linelist().copy()
        #Intensity Thresholding
        param_linelist_df = param_linelist_df[param_linelist_df['sw'] > self.threshold_intensity] #intensity thresholding
        #Cutdown linelist to parameters within specified simulation window of minimum and maximum of dataset
        dataset_min, dataset_max = (self.dataset.get_spectra_extremes())
        param_linelist_df = param_linelist_df[param_linelist_df['nu'] < (dataset_max + self.sim_window)]
        param_linelist_df = param_linelist_df[param_linelist_df['nu'] > (dataset_min - self.sim_window)]

        #delete parameters not partaining to species (where relevant)
        diluent_list = []
        for spectrum in self.dataset.spectra:
            for diluent in spectrum.Diluent:
                if diluent not in diluent_list:
                    diluent_list.append(diluent)
        num_nominal_temps, list_nominal_temps = self.dataset.get_number_nominal_temperatures()
        column_list = ['molec_id', 'local_iso_id','elower', 'nu', 'sw']        
        for diluent in diluent_list:
            column_list.append('gamma0_' + diluent)
            column_list.append('n_gamma0_' + diluent)
            column_list.append('delta0_' + diluent)
            column_list.append('n_delta0_' + diluent)
            column_list.append('SD_gamma_' + diluent)
            column_list.append('n_gamma2_' + diluent)
            column_list.append('SD_delta_' + diluent)
            column_list.append('n_delta2_' + diluent)
            column_list.append('nuVC_' + diluent)
            column_list.append('n_nuVC_' + diluent)
            column_list.append('eta_' + diluent)
            for nominal_temperature in list_nominal_temps:
                column_list.append('y_' + diluent + '_' + str(nominal_temperature)) ## Fix this so it has the nominal temperatures
        param_linelist_df = param_linelist_df[column_list]
        param_linelist_df = param_linelist_df.reset_index(drop = True)
        
        # Defines the Linecenter parameters in the event that the line center is held constant across all samples and is not
        ## Starting point is equal to the inital value
        order_nu = ['nu']
        param_linelist_df['nu_vary'] = len(param_linelist_df)*[False]
        param_linelist_df['nu_err'] = len(param_linelist_df)*[0]
        if self.nu_constrain:
            if vary_nu != {}:
                for molecule in vary_nu:
                    for isotope in vary_nu[molecule]:
                        param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) & (param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'nu_vary'] = (vary_nu[molecule][isotope])
        else:
            for spec in self.dataset.get_list_spectrum_numbers():
                param_linelist_df['nu_' + str(spec)] = param_linelist_df['nu'].values
                order_nu.append('nu_' + str(spec))
                param_linelist_df['nu_' + str(spec) + '_vary'] = len(param_linelist_df)*[False]
                param_linelist_df['nu_' + str(spec) + '_err'] = len(param_linelist_df)*[0]
                if vary_nu != {}:
                    for molecule in vary_nu:
                        for isotope in vary_nu[molecule]:
                            param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) & (param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'nu_' + str(spec) + '_vary'] = (vary_nu[molecule][isotope])             
        
        # Defines the linestrength in the even that the sw is held constant across all samples and not
        ## Starting point is equal to the initial value
        order_sw = ['sw']
        param_linelist_df['sw_vary'] = len(param_linelist_df)*[False]
        param_linelist_df['sw_err'] = len(param_linelist_df)*[0]
        if self.sw_constrain:
            if vary_sw != {}:
                 for molecule in vary_sw:
                    for isotope in vary_sw[molecule]:
                        param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'sw_vary'] = (vary_sw[molecule][isotope])
        else:
            for spec in self.dataset.get_list_spectrum_numbers():
                param_linelist_df['sw_' + str(spec)] = param_linelist_df['sw'].values
                order_sw.append('sw_' + str(spec))
                param_linelist_df['sw_' + str(spec) + '_vary'] = len(param_linelist_df)*[False]
                param_linelist_df['sw_' + str(spec) + '_err'] = len(param_linelist_df)*[0]
                if vary_sw != {}:
                    for molecule in vary_sw:
                        for isotope in vary_sw[molecule]:
                            param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'sw_' + str(spec) + '_vary'] = (vary_sw[molecule][isotope])
        #Loop through other parameters and then set things to 0 based on lineshape 
        order_gamma0 = []
        order_delta0 = []
        order_SD_gamma = []
        order_SD_delta = []
        order_nuVC = []
        order_eta = []
        order_linemixing = []
        for diluent in diluent_list:
            #Gamma0 option for constrain and not constrained
            order_gamma0.append('gamma0_' + diluent)
            param_linelist_df['gamma0_' + diluent + '_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['gamma0_' + diluent + '_err'] = len(param_linelist_df)*[0]
            if self.gamma0_constrain:
                if vary_gamma0 != {}:
                    for molecule in vary_gamma0:
                        for isotope in vary_gamma0[molecule]:
                            param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'gamma0_' +diluent + '_vary'] = (vary_gamma0[molecule][isotope])
            else:
                for spec in self.dataset.get_list_spectrum_numbers():
                    order_gamma0.append('gamma0_' +diluent + '_' +str(spec))
                    param_linelist_df['gamma0_' +diluent + '_' +str(spec)] = (param_linelist_df['gamma0_' + diluent].values)
                    param_linelist_df['gamma0_' + str(spec) + '_vary'] = len(param_linelist_df)*[False]
                    param_linelist_df['gamma0_' + str(spec) + '_err'] = len(param_linelist_df)*[0]
                    if vary_gamma0 != {}:
                        for molecule in vary_gamma0:
                            for isotope in vary_gamma0[molecule]:
                                param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'gamma0_' +diluent +'_'+str(spec) + '_vary'] = (vary_gamma0[molecule][isotope])
            
            order_gamma0.append('n_gamma0_' +diluent )
            #Delta0 option for constrain and not constrained
            order_delta0.append('delta0_' + diluent)
            param_linelist_df['delta0_' + diluent + '_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['delta0_' + diluent + '_err'] = len(param_linelist_df)*[0]
            if self.delta0_constrain:
                if vary_delta0 != {}:
                    for molecule in vary_delta0:
                        for isotope in vary_delta0[molecule]:
                            param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'delta0_' +diluent + '_vary'] = (vary_delta0[molecule][isotope])
            else:
                for spec in self.dataset.get_list_spectrum_numbers():
                    order_delta0.append('delta0_' +diluent + '_' +str(spec))
                    param_linelist_df['delta0_' +diluent + '_' +str(spec)] = (param_linelist_df['delta0_' + diluent].values)
                    param_linelist_df['delta0_' + str(spec) + '_vary'] = len(param_linelist_df)*[0]
                    param_linelist_df['delta0_' + str(spec) + '_err'] = len(param_linelist_df)*[0]
                    if vary_delta0 != {}:
                        for molecule in vary_delta0:
                            for isotope in vary_delta0[molecule]:
                                param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'delta0_' +diluent +'_'+str(spec) + '_vary'] = (vary_delta0[molecule][isotope])
            order_delta0.append('n_delta0_' +diluent ) 
            #SD Gamma option for constrain and not constrained
            order_SD_gamma.append('SD_gamma_' + diluent )
            param_linelist_df['SD_gamma_' + diluent + '_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['SD_gamma_' + diluent + '_err'] = len(param_linelist_df)*[0]
            if self.aw_constrain:
                if (self.lineprofile == 'VP') or (self.lineprofile == 'NGP'):
                    param_linelist_df.loc[:, 'SD_gamma_' + diluent] = 0
                else:
                    if vary_aw != {}:
                        for molecule in vary_aw:
                            for isotope in vary_aw[molecule]:
                                param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'SD_gamma_' +diluent + '_vary'] = (vary_aw[molecule][isotope])
            else:
                for spec in self.dataset.get_list_spectrum_numbers():
                    order_SD_gamma.append('SD_gamma_' +diluent + '_' +str(spec))
                    param_linelist_df['SD_gamma_' + str(spec) + '_vary'] = len(param_linelist_df)*[False]
                    param_linelist_df['SD_gamma_' + str(spec) + '_err'] = len(param_linelist_df)*[0]
                    if (self.lineprofile == 'VP') or (self.lineprofile == 'NGP'):
                        param_linelist_df.loc[:, 'SD_gamma_' +diluent + '_' +str(spec)] = 0
                    else:
                        param_linelist_df['SD_gamma_' +diluent + '_' +str(spec)] = (param_linelist_df['SD_gamma_' + diluent].values)
                        if vary_aw != {}:
                            for molecule in vary_aw:
                                for isotope in vary_aw[molecule]:
                                    param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'SD_gamma_' +diluent +'_'+str(spec) + '_vary'] = (vary_aw[molecule][isotope])
            order_SD_gamma.append('n_gamma2_' +diluent )
            #SD Delta option for constrain and not constrained
            order_SD_delta.append('SD_delta_' + diluent)
            param_linelist_df['SD_delta_' + diluent + '_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['SD_delta_' + diluent + '_err'] = len(param_linelist_df)*[0]
            if self.as_constrain:
                if (self.lineprofile == 'VP') or (self.lineprofile == 'NGP'):
                    param_linelist_df.loc[:, 'SD_delta_' + diluent] = 0
                else:
                    if vary_as != {}:
                        for molecule in vary_as:
                            for isotope in vary_as[molecule]:
                                param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'SD_delta_' +diluent + '_vary'] = (vary_aw[molecule][isotope])
            else:
                for spec in self.dataset.get_list_spectrum_numbers():
                    order_SD_delta.append('SD_delta_' +diluent + '_' +str(spec))
                    param_linelist_df['SD_delta_' + str(spec) + '_vary'] = len(param_linelist_df)*[False]
                    param_linelist_df['SD_delta_' + str(spec) + '_err'] = len(param_linelist_df)*[0]
                    if (self.lineprofile == 'VP') or (self.lineprofile == 'NGP'):
                        param_linelist_df.loc[:, 'SD_delta_' +diluent + '_' +str(spec)] = 0
                    else:
                        param_linelist_df['SD_delta_' +diluent + '_' +str(spec)] = (param_linelist_df['SD_deltaamma_' + diluent].values)

                        if vary_as != {}:
                            for molecule in vary_as:
                                for isotope in vary_as[molecule]:
                                    param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'SD_delta_' +diluent +'_'+str(spec) + '_vary'] = (vary_as[molecule][isotope])           
            order_SD_delta.append('n_delta2_' +diluent )
            #nuVC option for constrain and not constrained
            order_nuVC.append('nuVC_' + diluent)
            param_linelist_df['nuVC_' + diluent + '_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['nuVC_' + diluent + '_err'] = len(param_linelist_df)*[0]
            if self.nuVC_constrain:
                if (self.lineprofile == 'VP') or (self.lineprofile == 'SDVP'):
                    param_linelist_df.loc[:, 'nuVC_' + diluent ] = 0
                else:
                    if vary_nuVC != {}:
                        for molecule in vary_nuVC:
                            for isotope in vary_nuVC[molecule]:
                                param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'nuVC_' +diluent + '_vary'] = (vary_nuVC[molecule][isotope])
            else:
                for spec in self.dataset.get_list_spectrum_numbers():
                    order_nuVC.append('nuVC_' + diluent + '_'+str(spec))
                    param_linelist_df['nuVC_' + diluent + '_'+str(spec) + '_vary'] = len(param_linelist_df)*[False]
                    param_linelist_df['nuVC_' + diluent + '_'+ str(spec) + '_err'] = len(param_linelist_df)*[0]
                    if (self.lineprofile == 'VP') or (self.lineprofile == 'SDVP'):
                        param_linelist_df.loc[:, 'nuVC_' + diluent + '_'+str(spec)] = 0                       
                    else:
                        param_linelist_df['nuVC_' +diluent + '_' +str(spec)] = (param_linelist_df['nuVC_' + diluent].values)

                        if vary_nuVC != {}:
                            for molecule in vary_nuVC:
                                for isotope in vary_nuVC[molecule]:
                                    param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'nuVC_' +diluent +'_'+str(spec) + '_vary'] = (vary_nuVC[molecule][isotope])
            order_nuVC.append('n_nuVC_' +diluent )
            #eta option for constrain and not constrained
            order_eta.append('eta_' + diluent)
            param_linelist_df['eta_' + diluent + '_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['eta_' + diluent + '_err'] = len(param_linelist_df)*[0]
            if self.eta_constrain:
                if (self.lineprofile == 'VP') or (self.lineprofile == 'SDVP') or (self.lineprofile == 'NGP') or (self.lineprofile == 'SDNGP'):
                    param_linelist_df.loc[:, 'eta_' + diluent] = 0
                else:
                    if vary_eta != {}:
                        for molecule in vary_eta:
                            for isotope in vary_eta[molecule]:
                                param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'eta_' +diluent + '_vary'] = (vary_eta[molecule][isotope])
            else:
                for spec in self.dataset.get_list_spectrum_numbers():
                    order_eta.append('eta_' +diluent + '_' +str(spec))
                    param_linelist_df['eta_' + str(spec) + '_vary'] = len(param_linelist_df)*[False]
                    param_linelist_df['eta_' + str(spec) + '_err'] = len(param_linelist_df)*[0]
                    if (self.lineprofile == 'VP') or (self.lineprofile == 'SDVP') or (self.lineprofile == 'NGP') or (self.lineprofile == 'SDNGP'):
                        param_linelist_df.loc[:, 'eta_' +diluent + '_' +str(spec)] = 0                       
                    else:
                        param_linelist_df['eta_' +diluent + '_' +str(spec)] = (param_linelist_df['eta_' + diluent].values)
                        if vary_eta != {}:
                            for molecule in vary_eta:
                                for isotope in vary_eta[molecule]:
                                    param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'eta_' +diluent +'_'+str(spec) + '_vary'] = (vary_eta[molecule][isotope])
            # Linemixing
            
            for nominal_temp in list_nominal_temps:
                order_linemixing.append('y_' + diluent + '_'+ str(nominal_temp))
                param_linelist_df['y_' + diluent + '_'+ str(nominal_temp) + '_vary'] = len(param_linelist_df)*[False]
                param_linelist_df['y_' + diluent + '_'+ str(nominal_temp)+ '_err'] = len(param_linelist_df)*[0]
                if self.linemixing_constrain:
                    if not self.linemixing:
                        param_linelist_df.loc[:,'y_' + diluent + '_' + str(nominal_temp)] = 0
                    else:
                        if vary_linemixing != {}:
                            for molecule in vary_linemixing:
                                for isotope in vary_linmixing[molecule]:
                                    param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'y_' +diluent + '_'+ str(nominal_temp)+ '_vary'] = (vary_linemixing[molecule][isotope])
                else:
                    for spec in self.dataset.get_list_spectrum_numbers():
                        order_linemixing.append('y_' + diluent + '_'+ str(nominal_temp)+ '_'+ str(spec))
                        param_linelist_df['y_' +diluent + '_'+ str(nominal_temp) + '_'+ str(spec) + '_vary'] = len(param_linelist_df)*[False]
                        param_linelist_df['y_' +diluent + '_'+ str(nominal_temp) + '_'+ str(spec) + '_err'] = len(param_linelist_df)*[0]
                        if not self.linemixing:
                            param_linelist_df.loc[:,'y_' + diluent + '_'+str(nominal_temp) +'_' + str(spec)] = 0
                        else:
                            param_linelist_df['y_' +diluent + '_'+ str(nominal_temp)+'_' +str(spec)] = (param_linelist_df['y_' + diluent].values)
                            if vary_linemixing != {}:
                                for molecule in vary_linemixing:
                                    for isotope in vary_linemixing[molecule]:
                                        param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'y_' +diluent+ '_'+ str(nominal_temp) +'_'+str(spec) + '_vary'] = (vary_linemixing[molecule][isotope])


        #Temperature Dependence
        
        if num_nominal_temps > 1:
            #n_Gamma0
            param_linelist_df['n_gamma0'+diluent+'_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['n_gamma0_'+diluent+'_err'] = len(param_linelist_df)*[0]
            if vary_n_gamma0 != {}:
                for molecule in vary_n_gamma0:
                    for isotope in vary_n_gamma0[molecule]:
                        param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'n_gamma0_' +diluent + '_vary'] = (vary_n_gamma0[molecule][isotope])
            #n_Delta0
            param_linelist_df['n_delta0'+diluent+'_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['n_delta0_'+diluent+'_err'] = len(param_linelist_df)*[0]
            if vary_n_delta0 != {}:
                for molecule in vary_n_delta0:
                    for isotope in vary_n_delta0[molecule]:
                        param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'n_delta0_' +diluent + '_vary'] = (vary_n_delta0[molecule][isotope])
            #n_Gamma2
            param_linelist_df['n_gamma2'+diluent+'_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['n_gamma2_'+diluent+'_err'] = len(param_linelist_df)*[0]
            if not (self.lineprofile == 'VP') or  not (self.lineprofile == 'NGP') :
                if vary_n_gamma2 != {}:
                    for molecule in vary_n_gamma2:
                        for isotope in vary_n_gamma2[molecule]:
                            param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'n_gamma2_' +diluent + '_vary'] = (vary_n_gamma2[molecule][isotope])

            #n_Delta2
            param_linelist_df['n_delta2'+diluent+'_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['n_delta2_'+diluent+'_err'] = len(param_linelist_df)*[0]
            if not (self.lineprofile == 'VP') or  not (self.lineprofile == 'NGP') :
                if vary_n_delta2 != {}:
                    for molecule in vary_n_delta2:
                        for isotope in vary_n_delta2[molecule]:
                            param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'n_delta2_' +diluent + '_vary'] = (vary_n_delta2[molecule][isotope])
            #n_nuVC
            param_linelist_df['n_nuVC'+diluent+'_vary'] = len(param_linelist_df)*[False]
            param_linelist_df['n_nuVC_'+diluent+'_err'] = len(param_linelist_df)*[0]
            if not (self.lineprofile == 'VP') or  not (self.lineprofile == 'SDVP') :
                if vary_n_nuVC != {}:
                    for molecule in vary_n_nuVC:
                        for isotope in vary_n_nuVC[molecule]:
                            param_linelist_df.loc[(param_linelist_df['sw'] > self.fit_intensity) &(param_linelist_df['molec_id'] == molecule) & (param_linelist_df['local_iso_id'] == isotope), 'n_nuVC_' +diluent + '_vary'] = (vary_n_nuVC[molecule][isotope])
        
    
        #param_linelist_df = param_linelist_df.reindex(sorted(param_linelist_df.columns), axis=1)
        ordered_list = ['molec_id', 'local_iso_id','elower']
        for item in order_nu:
            ordered_list.append(item)
            ordered_list.append(item + '_err')
            ordered_list.append(item + '_vary')
        for item in order_sw:
            ordered_list.append(item)
            ordered_list.append(item + '_err')
            ordered_list.append(item + '_vary')
        for item in order_gamma0:
            ordered_list.append(item)
            if num_nominal_temps > 1:
                ordered_list.append(item + '_err')
                ordered_list.append(item + '_vary')
            else:
                if 'n_' != item[:2]:
                    ordered_list.append(item + '_err')
                    ordered_list.append(item + '_vary')
        for item in order_delta0:
            ordered_list.append(item)
            if num_nominal_temps > 1:
                ordered_list.append(item + '_err')
                ordered_list.append(item + '_vary')
            else:
                if 'n_' != item[:2]:
                    ordered_list.append(item + '_err')
                    ordered_list.append(item + '_vary')
        for item in order_nuVC:
            ordered_list.append(item)
            if num_nominal_temps > 1:
                ordered_list.append(item + '_err')
                ordered_list.append(item + '_vary')
            else:
                if 'n_' != item[:2]:
                    ordered_list.append(item + '_err')
                    ordered_list.append(item + '_vary')
        for item in order_SD_gamma:
            ordered_list.append(item)
            if num_nominal_temps > 1:
                ordered_list.append(item + '_err')
                ordered_list.append(item + '_vary')
            else:
                if 'n_' != item[:2]:
                    ordered_list.append(item + '_err')
                    ordered_list.append(item + '_vary')
        for item in order_SD_delta:
            ordered_list.append(item)
            if num_nominal_temps > 1:
                ordered_list.append(item + '_err')
                ordered_list.append(item + '_vary')
            else:
                if 'n_' != item[:2]:
                    ordered_list.append(item + '_err')
                    ordered_list.append(item + '_vary')
        for item in order_eta:
            ordered_list.append(item)
            ordered_list.append(item + '_err')
            ordered_list.append(item + '_vary')
        for item in order_linemixing:
            ordered_list.append(item)
            ordered_list.append(item + '_err')
            ordered_list.append(item + '_vary')
            
        
        param_linelist_df = param_linelist_df[ordered_list]
        param_linelist_df.to_csv(self.param_linelist_savename + '.csv')
        return param_linelist_df
 

    def generate_fit_baseline_linelist(self, vary_baseline = True, vary_concentration = {7:True, 1:False}, vary_xshift = False, 
                                      vary_etalon_amp= False, vary_etalon_freq= False, vary_etalon_phase= False):
        base_linelist_df = self.get_base_linelist().copy()
        parameters =  (list(base_linelist_df))
        for param in parameters:
            base_linelist_df[param + '_err'] = len(base_linelist_df)*[0]
            if 'baseline' in param:
                base_linelist_df[param + '_vary'] = len(base_linelist_df)*[True]
            if 'Concentration' in param:
                for molecule in vary_concentration:
                    if (ISO[(molecule, 1)][4]) in param:
                        base_linelist_df[param + '_vary'] = len(base_linelist_df)*[(vary_concentration[molecule])]
                        base_linelist_df[param + '_err'] = len(base_linelist_df)*[0]
            if 'x_shift' in param:
                base_linelist_df[param + '_vary'] = len(base_linelist_df)*[(vary_xshift)]
                base_linelist_df[param + '_err'] = len(base_linelist_df)*[0]
            if 'amp' in param:
                base_linelist_df[param + '_vary'] = len(base_linelist_df)*[(vary_etalon_amp)]
                base_linelist_df[param + '_err'] = len(base_linelist_df)*[0]
            if 'freq' in param:
                base_linelist_df[param + '_vary'] = len(base_linelist_df)*[(vary_etalon_freq)]
                base_linelist_df[param + '_err'] = len(base_linelist_df)*[0]
            if 'phase' in param:
                base_linelist_df[param + '_vary'] = len(base_linelist_df)*[(vary_etalon_phase)]  
                base_linelist_df[param + '_err'] = len(base_linelist_df)*[0] 
        base_linelist_df = base_linelist_df.reindex(sorted(base_linelist_df.columns), axis=1)
        base_linelist_df.to_csv(self.base_linelist_savename + '.csv')
        return base_linelist_df

    
class Edit_Fit_Param_Files:
    def __init__(self, base_linelist_file, param_linelist_file, new_base_linelist_file = None, new_param_linelist_file = None):
        self.base_linelist_file = base_linelist_file
        self.param_linelist_file = param_linelist_file
        if new_base_linelist_file == None:
            self.new_base_linelist_file = base_linelist_file
        else:
            self.new_base_linelist_file = new_base_linelist_file
        if new_param_linelist_file == None:
            self.new_param_linelist_file = new_param_linelist_file
        else:
            self.new_param_linelist_file = new_param_linelist_file
    
    def edit_generated_baselist(self):
        base_linelist_df = pd.read_csv(self.base_linelist_file + '.csv', index_col = 0)
        baseline_widget = qgrid.show_grid(base_linelist_df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
        return baseline_widget
    def save_editted_baselist(self, baseline_widget):
        base_linelist_df = baseline_widget.get_changed_df()
        base_linelist_df.to_csv(self.new_base_linelist_file + '.csv')
        return base_linelist_df
    def edit_generated_paramlist(self):
        param_linelist_df = pd.read_csv(self.param_linelist_file + '.csv', index_col = 0)
        param_widget = qgrid.show_grid(param_linelist_df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
        return param_widget
    def save_editted_paramlist(self, param_widget):
        param_linelist_df = param_widget.get_changed_df()
        param_linelist_df.to_csv(self.new_param_linelist_file + '.csv')
        return base_linelist_df
        
        
        

FITPARAMS = Generate_FitParam_File(SPECTRA, PARAM_LINELIST, BASE_LINELIST)

FITPARAMS.generate_fit_param_linelist_from_linelist()




,molec_id,local_iso_id,elower,nu,nu_err,nu_vary,sw,sw_err,sw_vary,sw_1,...,SD_delta_air,SD_delta_air_err,SD_delta_air_vary,n_delta2_air,eta_air,eta_air_err,eta_air_vary,y_air_296,y_air_296_err,y_air_296_vary
0,7,3,1212.2838,13004.62208,0,False,5.060000e-29,0,False,5.060000e-29,...,0,0,False,0,0,0,False,0,0,False
1,7,2,1259.1230,13005.26291,0,False,1.920000e-28,0,False,1.920000e-28,...,0,0,False,0,0,0,False,0,0,False
2,7,1,2097.0889,13005.34522,0,False,9.090000e-28,0,False,9.090000e-28,...,0,0,False,0,0,0,False,0,0,False
3,7,1,1989.9904,13005.67376,0,False,1.470000e-27,0,False,1.470000e-27,...,0,0,False,0,0,0,False,0,0,False
4,7,3,1210.5248,13006.38112,0,False,4.880000e-29,0,False,4.880000e-29,...,0,0,False,0,0,0,False,0,0,False
5,7,1,2213.3478,13006.73085,0,False,5.710000e-28,0,False,5.710000e-28,...,0,0,False,0,0,0,False,0,0,False
6,7,1,2094.9441,13007.49001,0,False,9.820000e-28,0,False,9.820000e-28,...,0,0,False,0,0,0,False,0,0,False
7,7,1,2340.8740,13007.70125,0,False,3.370000e-28,0,False,3.370000e-28,...,0,0,False,0,0,0,False,0,0,False
8,7,1,2790.8388,13008.08467,0,False,4.760000e-29,0,False,4.760000e-29,...,0,0,False,0,0,0,False,0,0,False
9,7,1,2479.6468,13008.25289,0,False,1.860000e-28,0,False,1.860000e-28,...,0,0,False,0,0,0,False,0,0,False


In [171]:
edit_files = Edit_Fit_Param_Files('Baseline_LineList', 'Parameter_LineList')
edit_files.edit_generated_baselist()

QgridWidget(grid_options={'highlightSelectedCell': False, 'boldIndex': True, 'autoEdit': False, 'fullWidthRows…

In [299]:
#Generate a Fit Class
# Define the model 

# Enter data for fitting
# input x, y columns?

def hasNumbers(inputString):
    has_digits = False
    for char in inputString:
        if char.isdigit():
            return True
    return False

class Fit_DataSet:
    def __init__(self, dataset, base_linelist_file, param_linelist_file, fit_intensity = 1e-25,
                baseline_limit = True, baseline_limit_percent = 10, 
                concentration_limit = True, concentration_limit_percent = 10, 
                etalon_limit = False, etalon_limit_percent = 10, #phase is constrained to +/- 2pi, 
                x_shift_limit = True, x_shift_limit_magnitude = 0.1, 
                nu_limit = True, nu_limit_magnitude = 0.1, 
                sw_limit = False, sw_limit_percent = 10, 
                gamma0_limit = True, gamma0_limit_percent = 10, n_gamma0_limit= True, n_gamma0_limit_percent = 10, 
                delta0_limit = True, delta0_limit_percent = 10, n_delta0_limit = True, n_delta0_limit_percent = 10, 
                SD_gamma_limit = True, SD_gamma_limit_percent = 10, n_gamma2_limit = True, n_gamma2_limit_percent = 10, 
                SD_delta_limit = True, SD_delta_limit_percent = 10, n_delta2_limit = True, n_delta2_limit_percent = 10, 
                nuVC_limit = True, nuVC_limit_percent = 10, n_nuVC_limit = True, n_nuVC_limit_percent = 10, 
                eta_limit = True, eta_limit_percent = 10, linemixing_limit = True, linemixing_limit_percent = 10):
        self.dataset = dataset
        self.base_linelist_file = base_linelist_file
        self.param_linelist_file = param_linelist_file
        self.fit_intensity = fit_intensity
        self.baseline_limit = baseline_limit
        self.baseline_limit_percent = baseline_limit_percent
        self.etalon_limit = etalon_limit
        self.etalon_limit_percent = etalon_limit_percent
        self.concentration_limit = concentration_limit
        self.concentration_limit_percent = concentration_limit_percent
        self.etalon_limit = etalon_limit
        self.etalon_limit_percent = etalon_limit_percent
        self.x_shift_limit = x_shift_limit
        self.x_shift_limit_magnitude = x_shift_limit_magnitude
        self.nu_limit = nu_limit
        self.nu_limit_magnitude = nu_limit_magnitude
        self.sw_limit = sw_limit
        self.sw_limit_percent = sw_limit_percent
        self.gamma0_limit = gamma0_limit
        self.gamma0_limit_percent = gamma0_limit_percent
        self.n_gamma0_limit = n_gamma0_limit
        self.n_gamma0_limit_percent = n_gamma0_limit_percent
        self.delta0_limit = delta0_limit
        self.delta0_limit_percent = delta0_limit_percent
        self.n_delta0_limit = n_delta0_limit
        self.n_delta0_limit_percent = n_delta0_limit_percent
        self.SD_gamma_limit = SD_gamma_limit
        self.SD_gamma_limit_percent = SD_gamma_limit_percent
        self.n_gamma2_limit = n_gamma2_limit
        self.n_gamma2_limit_percent = n_gamma2_limit_percent
        self.SD_delta_limit = SD_gamma_limit
        self.SD_delta_limit_percent = SD_delta_limit_percent
        self.n_delta2_limit = n_delta2_limit
        self.n_delta2_limit_percent = n_delta2_limit_percent
        self.nuVC_limit = nuVC_limit
        self.nuVC_limit_percent = nuVC_limit_percent
        self.n_nuVC_limit = n_nuVC_limit
        self.n_nuVC_limit_percent = n_nuVC_limit_percent
        self.eta_limit = eta_limit
        self.eta_limit_percent = eta_limit_percent
        self.linemixing_limit = linemixing_limit
        self.linemixing_limit_percent = linemixing_limit_percent
    def generate_params(self):
        params = Parameters()
        
        #Baseline Parameters
        baseline_list = pd.read_csv(self.base_linelist_file + '.csv', index_col = 0)
        baseline_parameters = []
        for base_param in list(baseline_list):
            if ('_vary' not in base_param) and ('_err' not in base_param):
                baseline_parameters.append(base_param)
        for spec_num in baseline_list.index.values:
            for base_param in baseline_parameters:
                if baseline_list.loc[int(spec_num)][base_param] == 0:
                    params.add(base_param + '_'+str(int(spec_num)), baseline_list.loc[int(spec_num)][base_param], baseline_list.loc[int(spec_num)][base_param + '_vary'])
                elif ('Concentration' in base_param) and self.concentration_limit:
                    params.add(base_param + '_'+str(int(spec_num)), baseline_list.loc[int(spec_num)][base_param], baseline_list.loc[int(spec_num)][base_param + '_vary'], 
                              min = (1 - (self.concentration_limit_percent / 100))*baseline_list.loc[int(spec_num)][base_param], 
                              max = (1 + (self.concentration_limit_percent / 100))*baseline_list.loc[int(spec_num)][base_param])
                elif ('baseline' in base_param) and self.baseline_limit:
                    params.add(base_param + '_'+str(int(spec_num)), baseline_list.loc[int(spec_num)][base_param], baseline_list.loc[int(spec_num)][base_param + '_vary'], 
                              min = (1 - (self.baseline_limit_percent / 100))*baseline_list.loc[int(spec_num)][base_param], 
                              max = (1 + (self.baseline_limit_percent / 100))*baseline_list.loc[int(spec_num)][base_param])
                elif ('etalon_' in base_param) and self.etalon_limit and ('phase' not in base_param):
                    params.add(base_param + '_'+str(int(spec_num)), baseline_list.loc[int(spec_num)][base_param], baseline_list.loc[int(spec_num)][base_param + '_vary'], 
                              min = (1 - (self.etalon_limit_percent / 100))*baseline_list.loc[int(spec_num)][base_param], 
                              max = (1 + (self.etalon_limit_percent / 100))*baseline_list.loc[int(spec_num)][base_param])
                elif ('etalon_' in base_param) and self.etalon_limit and ('phase' in base_param):
                    params.add(base_param + '_'+str(int(spec_num)), baseline_list.loc[int(spec_num)][base_param], baseline_list.loc[int(spec_num)][base_param + '_vary'], 
                              min = -2*np.pi, max = 2*np.pi)
                elif ('x_shift' in base_param) and self.x_shift_limit:
                    params.add(base_param + '_'+str(int(spec_num)), baseline_list.loc[int(spec_num)][base_param], baseline_list.loc[int(spec_num)][base_param + '_vary'], 
                              min = (baseline_list.loc[int(spec_num)][base_param] - x_shift_limit_magnitude), 
                              max = 1 + baseline_list.loc[int(spec_num)][base_param])   
                else:
                    params.add(base_param + '_'+str(int(spec_num)), baseline_list.loc[int(spec_num)][base_param], baseline_list.loc[int(spec_num)][base_param + '_vary'])
        
        #Lineshape parameters
        lineparam_list = pd.read_csv(self.param_linelist_file + '.csv', index_col = 0)
        linelist_params = []
        
        for line_param in list(lineparam_list):
            if (self.dataset.get_number_nominal_temperatures()[0]) == 1:
                if ('_vary' not in line_param) and ('_err' not in line_param) and (line_param != 'molec_id') and (line_param != 'local_iso_id') and (line_param != 'elower') and ('n_' not in line_param):
                    linelist_params.append(line_param)
            else:
                if ('_vary' not in line_param) and ('_err' not in line_param) and (line_param != 'molec_id') and (line_param != 'local_iso_id') and (line_param != 'elower'):
                    linelist_params.append(line_param)
        diluent_list = []
        for spectrum in self.dataset.spectra:
            for diluent in spectrum.Diluent:
                if diluent not in diluent_list:
                    diluent_list.append(diluent)
        nu_constrain = True
        sw_constrain = True
        gamma0_constrain = True
        delta0_constrain = True
        SD_gamma_constrain = True
        SD_delta_constrain = True
        nuVC_constrain = True
        eta_constrain = True
        linemix_constrain = True
        
        if ((sum('nu' in param for param in linelist_params)) - (sum('nuVC' in param for param in linelist_params))) > 1:
            nu_constrain = False
        if (sum('sw' in param for param in linelist_params)) > 1:
            sw_constrain = False
        if (self.dataset.get_number_nominal_temperatures()[0]) == 1:
            if (sum('gamma0' in param for param in linelist_params)) >  len(diluent_list):
                gamma0_constrain = False
            if (sum('delta0' in param for param in linelist_params)) >  len(diluent_list):
                delta0_constrain = False
            if (sum('SD_gamma' in param for param in linelist_params)) >  len(diluent_list):
                SD_gamma_constrain = False
            if (sum('SD_delta' in param for param in linelist_params)) >  len(diluent_list):
                SD_delta_constrain = False
            if (sum('nuVC' in param for param in linelist_params)) >  len(diluent_list):
                nuVC_constrain = False
            if (sum('eta' in param for param in linelist_params)) >  len(diluent_list):
                eta_constrain = False 
        else:
            if (sum('gamma0' in param for param in linelist_params)) >  len(diluent_list) + 1:
                gamma0_constrain = False
            if (sum('delta0' in param for param in linelist_params)) >  len(diluent_list) + 1:
                delta0_constrain = False
            if (sum('SD_gamma' in param for param in linelist_params)) >  len(diluent_list) + 1:
                SD_gamma_constrain = False
            if (sum('SD_delta' in param for param in linelist_params)) >  len(diluent_list) + 1:
                SD_delta_constrain = False
            if (sum('nuVC' in param for param in linelist_params)) >  len(diluent_list) + 1:
                nuVC_constrain = False
            if (sum('eta' in param for param in linelist_params)) >  len(diluent_list) + 1:
                eta_constrain = False 
        if (sum('y' in param for param in linelist_params)) >  len(diluent_list)*(self.dataset.get_number_nominal_temperatures()[0]):
            linemix_constrain = False
        linemix_terms_constrained = []
        for diluent in diluent_list:
            for temperature in (self.dataset.get_number_nominal_temperatures()[1]):
                linemix_terms_constrained.append('y_'+diluent + '_' + str(temperature))
        
        for spec_line in lineparam_list.index.values:
            if lineparam_list.loc[spec_line]['sw'] >= self.fit_intensity:
                for line_param in linelist_params:
                    #NU
                    if line_param == 'nu' and nu_constrain:
                        if self.nu_limit:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                      min = lineparam_list.loc[spec_line][line_param] - self.nu_limit_magnitude, 
                                      max = lineparam_list.loc[spec_line][line_param] + self.nu_limit_magnitude)
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif (line_param != 'nu') and ('nu' in line_param) and ('nuVC' not in line_param) and (not nu_constrain):
                        if self.nu_limit:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                      min = lineparam_list.loc[spec_line][line_param] - self.nu_limit_magnitude, 
                                      max = lineparam_list.loc[spec_line][line_param] + self.nu_limit_magnitude)
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    #SW
                    elif line_param == 'sw' and sw_constrain:
                        if self.sw_limit:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.sw_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.sw_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif (line_param != 'sw') and ('sw' in line_param) and (not sw_constrain):
                        if self.sw_limit:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.sw_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.sw_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    #GAMMA0
                    elif ('gamma0_' in line_param) and ('n_' not in line_param) and (gamma0_constrain) and (not hasNumbers(line_param.replace("gamma0_", ''))):
                        if self.gamma0_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.gamma0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.gamma0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                                
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('gamma0_' in line_param) and ('n_' not in line_param) and (not gamma0_constrain) and (hasNumbers(line_param.replace("gamma0_", ''))):
                        if self.gamma0_limit and lineparam_list.loc[spec_line][line_param] != 0:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.gamma0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.gamma0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('n_gamma0' in line_param):
                        if self.n_gamma0_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.n_gamma0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.n_gamma0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    #DELTA0
                    elif ('delta0' in line_param) and ('n_' not in line_param) and (delta0_constrain) and (not hasNumbers(line_param.replace("delta0", ''))):
                        if self.delta0_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.delta0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.delta0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('delta0_' in line_param) and ('n_' not in line_param) and (not delta0_constrain) and (hasNumbers(line_param.replace("delta0_", ''))):
                        if self.delta0_limit and lineparam_list.loc[spec_line][line_param] != 0:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.delta0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.delta0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('n_delta0' in line_param):
                        if self.n_delta0_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.n_delta0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.n_delta0_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    #SD Gamma
                    elif ('SD_gamma' in line_param) and (SD_gamma_constrain) and (not hasNumbers(line_param.replace("SD_gamma", ''))):
                        if self.SD_gamma_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.SD_gamma_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.SD_gamma_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('SD_gamma' in line_param) and (not SD_gamma_constrain) and (hasNumbers(line_param.replace("SD_gamma", ''))):
                        if self.SD_gamma_limit and lineparam_list.loc[spec_line][line_param] != 0:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.SD_gamma_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.SD_gamma_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('n_gamma2' in line_param):
                        if self.n_gamma2_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.n_gamma2_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.n_gamma2_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    #SD Delta
                    elif ('SD_delta' in line_param) and (SD_delta_constrain) and (not hasNumbers(line_param.replace("SD_delta", ''))):
                        if self.SD_delta_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.SD_delta_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.SD_delta_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('SD_delta' in line_param) and (not SD_delta_constrain) and (hasNumbers(line_param.replace("SD_delta", ''))):
                        if self.SD_delta_limit and lineparam_list.loc[spec_line][line_param] != 0:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.SD_delta_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.SD_delta_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('n_delta2' in line_param):
                        if self.n_delta2_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.n_delta2_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.n_delta2_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    #nuVC
                    elif ('nuVC' in line_param) and ('n_' not in line_param) and (nuVC_constrain) and (not hasNumbers(line_param.replace("nuVC", ''))):
                        if self.nuVC_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.nuVC_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.nuVC_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('nuVC_' in line_param) and ('n_' not in line_param) and (not nuVC_constrain) and (hasNumbers(line_param.replace("nuVC_", ''))):
                        if self.nuVC_limit and lineparam_list.loc[spec_line][line_param] != 0:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.nuVC_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.nuVC_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('n_nuVC' in line_param):
                        if self.n_nuVC_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.n_nuVC_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.n_nuVC_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])                    
                    #eta
                    elif ('eta_' in line_param) and (eta_constrain) and (not hasNumbers(line_param.replace("eta", ''))):
                        if self.eta_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.eta_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.eta_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('eta_' in line_param) and (not eta_constrain) and (hasNumbers(line_param.replace("eta_", ''))):
                        if self.eta_limit and lineparam_list.loc[spec_line][line_param] != 0:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.eta_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.eta_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary']) 
                    # linemixing
                    elif ('y_' in line_param) and (linemix_constrain) and (line_param in linemix_terms_constrained):
                        if self.linemixing_limit and lineparam_list.loc[spec_line][line_param] != 0:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.linemixing_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.linemixing_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                            params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'])
                    elif ('y_' in line_param) and (not linemix_constrain) and (not line_param in linemix_terms_constrained):
                        if self.linemixing_limit and lineparam_list.loc[spec_line][line_param] != 0:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary'], 
                                  min = (1 - (self.linemixing_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param], 
                                  max = (1 + (self.linemixing_limit_percent / 100))*lineparam_list.loc[int(spec_line)][line_param])
                        else:
                             params.add(line_param + '_' + 'line_' + str(spec_line), lineparam_list.loc[spec_line][line_param], lineparam_list.loc[spec_line][line_param + '_vary']) 
        return (params)
    def simulation_model(self, params, wave_space = 0.01, wing_cutoff = 50, wing_wavenumbers = 50):
        baseline_params = []
        for param in (list(params.valuesdict().keys())):
            if ('Concentration' in param) or ('baseline' in param) or ('etalon' in param) or ('x_shift' in param):
                baseline_params.append(param)
        lineparam_list = pd.read_csv(self.param_linelist_file + '.csv', index_col = 0)
        
        for spectrum in self.dataset.spectra:          
            spectrum_number = spectrum.spectrum_number
            pressure = spectrum.pressure
            temperature = spectrum.temperature
            wave_min = np.min(spectrum.wavenumber)
            wave_max = np.max(spectrum.wavenumber)
            concentration = spectrum.concentration
            Diluent = spectrum.Diluent
            natural_abundance = spectrum.natural_abundance
            abundance_ratio_MI = spectrum.abundance_ratio_MI
            nominal_temp = spectrum.nominal_temperature
            columns = ['molec_id', 'local_iso_id', 'elower', 'nu', 'sw']
            
            for species in Diluent:
                columns.append('gamma0_' + species)
                columns.append('n_gamma0_'+ species)
                columns.append('delta0_' + species)
                columns.append('n_delta0_'+ species)
                columns.append('SD_gamma_' + species)
                columns.append('n_gamma2_'+ species )
                columns.append('SD_delta_' + species)
                columns.append('n_delta2_' + species)
                columns.append('nuVC_' + species)
                columns.append('n_nuVC_' + species)
                columns.append('eta_' + species)
                columns.append('y_' + species + '_' + str(nominal_temp))
            rename_dictionary = {}
            for column in lineparam_list:
                if ('vary' not in column) and ('err' not in column):
                    if (column + '_' + str(spectrum_number)) in lineparam_list:
                        columns = [(column + '_'  + str(spectrum_number))if x==column else x for x in columns]
                        rename_dictionary[(column + '_'  + str(spectrum_number))] = column
            linelist_for_sim = lineparam_list[columns]
            
            # Replace relevant column names with parameter values
            linelist_for_sim=linelist_for_sim.rename(columns = rename_dictionary)
            
            
                    
            print (linelist_for_sim.head())   
                    
            
                
        
        
        
        
        
        
            
        
        
        #print (lineparam_list)
        
        
        #Determine constraints
        
        #print (linelist_params)
        # For each spectra will need to go through and grab either 
        '''
        for spectrum in self.dataset.spectra:
        
            # Generate a subset linelist
            
            # set values in that linelist to correspond to parameter values
            
            
            #print (nominal_temperature)
        '''

         
            
            #lineparam_list = pd.read_csv(self.param_linelist_file + '.csv', index_col = 0)
            
            #print (lineparam_list.head())
        
        
        '''
        HTP_from_DF_select(linelist, wave_min, wave_max,wave_space = wave_space, wing_cutoff = wing_cutoff, wing_wavenumbers = wing_wavenumbers, 
                pressure = 1, temperature = 296, concentration = {}, 
                natural_abundance = True, abundance_ratio_MI = {},  Diluent = {}, diluent = 'air')
        '''
        
        #print (params)
        
        
        
        
              
        
        
        
        
fit_data = Fit_DataSet(SPECTRA,'Baseline_LineList', 'Parameter_LineList')
params = fit_data.generate_params()
#params.pretty_print()
fit_data.simulation_model(params)



   molec_id  local_iso_id     elower           nu            sw  gamma0_air  \
0         7             3  1212.2838  13004.62208  5.060000e-29      0.0332   
1         7             2  1259.1230  13005.26291  1.920000e-28      0.0322   
2         7             1  2097.0889  13005.34522  9.090000e-28      0.0417   
3         7             1  1989.9904  13005.67376  1.470000e-27      0.0434   
4         7             3  1210.5248  13006.38112  4.880000e-29      0.0332   

   n_gamma0_air  delta0_air  n_delta0_air  SD_gamma_air  n_gamma2_air  \
0          0.63     -0.0092           0.0             0          0.63   
1          0.63     -0.0092           0.0             0          0.63   
2          0.64     -0.0073           0.0             0          0.64   
3          0.65     -0.0078           0.0             0          0.65   
4          0.63     -0.0091           0.0             0          0.63   

   SD_delta_air  n_delta2_air  nuVC_air  n_nuVC_air  eta_air  y_air_296  
0           

In [247]:
def sim_model(params, x, data, pressure_list,DF, x_min, fit_column, x_column, linelist):
    
    #Line parameters from linelist
    for i in range(0, len(linelist)):
        if (linelist[i]['nu'] > (DF[x_column].min() - wave_range)) and (linelist[i]['nu'] < (DF[x_column].max() + wave_range)):                                                                    
            if linelist[i]['sw'] >= Fit_Intensity: #should this be defined here
                linelist.loc[i, 'nu'] = np.float(params['nu_%i'%(i)])
                linelist.loc[i, 'sw'] = np.float(params['sw_%i'%(i)])
                for species in Diluent:
                    #Gamma0
                    linelist.loc[i, 'gamma0_%s'%(species)] = np.float(params['gamma0_%s_%i'%(species, i)]) 
                    linelist.loc[i, 'n_gamma0_%s'%(species)] = np.float(params['n_gamma0_%s_%i'%(species, i)]) 
                    #Delta0
                    linelist.loc[i, 'delta0_%s'%(species)] = np.float(params['delta0_%s_%i'%(species, i)]) 
                    linelist.loc[i, 'n_delta0_%s'%(species)] = np.float(params['n_delta0_%s_%i'%(species, i)]) 
                    #SD Gamma parameterized as aw
                    linelist.loc[i, 'SD_gamma_%s'%(species)] = np.float(params['SD_gamma_%s_%i'%(species, i)]) 
                    linelist.loc[i, 'n_gamma2_%s'%(species)] = np.float(params['n_gamma2_%s_%i'%(species, i)])
                    # SD Delta parameterizes as as
                    linelist.loc[i, 'SD_delta_%s'%(species)] = np.float(params['SD_delta_%s_%i'%(species, i)]) 
                    linelist.loc[i, 'n_delta2_%s'%(species)] = np.float(params['n_delta2_%s_%i'%(species, i)]) 
                    # nuVC
                    linelist.loc[i, 'nuVC_%s'%(species)] = np.float(params['nuVC_%s_%i'%(species, i)]) 
                    linelist.loc[i, 'n_nuVC_%s'%(species)] = np.float(params['n_nuVC_%s_%i'%(species, i)])
                    #eta
                    linelist.loc[i, 'eta_%s'%(species)] = np.float(params['eta_%s_%i'%(species, i)])
                    #line-mixing
                    linelist.loc[i, 'y_%s'%(species)] = np.float(params['y_%s_%i'%(species, i)])

    total_simulated = []
    total_residuals = []
    
    for spectrum in Dataset.spectra():
        for i in range(0, len(linelist)):
            if (linelist[i]['nu'] > (DF[x_column].min() - wave_range)) and (linelist[i]['nu'] < (DF[x_column].max() + wave_range)): 
                if linelist[i]['sw'] >= Fit_Intensity:
                    linelist[i]['sw'] = np.float(params['y_factor_%i_%i'%(i, pressure_num)])*np.float(params['sw_%i'%(i)])
        

    for pressure_num in pressure_list:
        for i in range(0, length(linelist)): 
            if (linelist[i]['nu'] > (DF[x_column].min() - wave_range)) and (linelist[i]['nu'] < (DF[x_column].max() + wave_range)): 
                if linelist[i]['sw'] >= Fit_Intensity:
                    linelist[i]['sw'] = np.float(params['y_factor_%i_%i'%(i, pressure_num)])*np.float(params['sw_%i'%(i)])

        ##TODO baseline + etalons
        #TODO Define concentration and abundance for file
        
        x_shift = np.float(params['x_shift_' + str(pressure_num)])
        
        x_pressure = DF[(DF['Filter'] ==0) ][x_column].values
        x_min_pressure =  DF[(DF['Filter'] ==0) ][x_column].min()
        y = DF[(DF['Filter'] ==0) ][fit_column].values
        
        ##TODO make diluent come through and natural abundance, and concentration
        natural_abundance = True
        #abundance_ratio_MI = {}
        #concentration = {8:0.2}
        #Diluent = {}
        #diluent = 'air'
        
        fit_nu, fit_coef = HTP_from_DF_select(linelist, x_pressure.min() - 1, x_pressure.max()+1 , pressure = DF['Cavity Pressure /Torr'].mean() / 760, temperature = DF['Cavity Temperature Side 2 /C'].mean() + 273.15,
                               concentration = { 7 : 0.2072}, natural_abundance = True, diluent = 'air', IntensityThreshold = IntensityThreshold)
        
        fit_coef *= 1e6
        fit_nu += x_shift
        
        etalons = etalon_model(params, x_pressure - np.float(x_min_pressure) ,pressure_num ,DF)
        baseline = linear_model(params, x_pressure - np.float(x_min_pressure) , pressure_num, DF)


        simulated_spectra = fit_coef +   baseline +etalons 
        residuals = simulated_spectra - y
       
        total_simulated = np.append(total_simulated, simulated_spectra)
        total_residuals = np.append(total_residuals, residuals)

    total_residuals = np.asarray(total_residuals)
    return total_residuals 


In [7]:
#Define Model
#Fetch the Hitran File
hapi = r'C:\Users\ema3\Documents\Python Scripts\HAPI' # Location of the Summary Data File
os.chdir(hapi)



os.chdir(hapi)


# allowing the outside lines to float is key!  But you might want to do just the main lines and then let them float.  
# also don't float the aw term

###LINE SHAPE PARAMETERS

x_column = 'Wavenumber'
fit_column = 'Alpha'


lineshape = 'SDVP' #VP, NGP, NGP-LM, SDVP-LM, SDNGP, SDNGP-LM


DBfromsource = True
center_vary = False
intensity_vary = True
width_vary = True
shift_vary = True
sd_vary = True
nuVC_vary = False
y_vary = False

fix_temp_dep = False
width_temp_vary = False
shift_temp_vary = False
sd_temp_vary = False
nuVC_temp_vary = False

Filter_save = True

### BASELINE AND ETALONS

copy_etalon = False
lock_etalons =  False
etalon_1_on =  True
etalon_2_on = True
etalon_3_on = False
etalon_4_on = False

lock_baseline = False
baseline_order =2 #(0, 1, 2)
baseline_vary = True
xshift_vary = False
x_shift = 0

#etalon_window = [0]
#etalon_mirror = [0, 1]

etalon_mirror = [0.001455, 25.07147*10**9 / 29979245800] 
etalon_window = [ 6.2325e-04]


# Define Background and Etalon Parameters
params.add('per_O2', percent_oxygen, vary = False)
for pressure_num in pressure_list:
    params.add('x_shift_%i'%(pressure_num),x_shift, vary = xshift_vary , min = -0.05, max = 0.05) #-0.005230285

    #Baseline Parameters
    if baseline_order == 2:
        params.add('y_a_%i'%(pressure_num), 0, vary = baseline_vary)
        params.add('y_b_%i'%(pressure_num), 0, vary = baseline_vary, min = -1, max = 1)
        params.add('y_c_%i'%(pressure_num), 0, vary = baseline_vary, min = -1, max = 1)
    if baseline_order == 1:
        params.add('y_a_%i'%(pressure_num), 2.621131, vary = baseline_vary)
        params.add('y_b_%i'%(pressure_num), 0.000072, vary = baseline_vary, min = -1, max = 1)
        params.add('y_c_%i'%(pressure_num), 0, vary = False, min = -1, max = 1)
    if baseline_order==0:
        params.add('y_a_%i'%(pressure_num), 0, vary = baseline_vary)
        params.add('y_b_%i'%(pressure_num), 0, vary = False, min = -1, max = 1)
        params.add('y_c_%i'%(pressure_num), 0, vary = False, min = -1, max = 1)

    # Define Etalon Parameters
    if etalon_1_on == False:  
        params.add('amp_1_%i'%(pressure_num),0, vary=False)
        params.add('freq_1_%i'%(pressure_num), 1 / etalon_mirror[1], vary = False, min = (1 / etalon_mirror[1])*0.8, max = (1 / etalon_mirror[1])*1.2)
        params.add('phase_1_%i'%(pressure_num),0, vary = False)
        #params.add('alpha_1_%i'%(pressure_num), 1, vary = False)
    elif etalon_1_on == True:
        params.add('amp_1_%i'%(pressure_num),etalon_mirror[0], vary=True, min = 0.5*etalon_mirror[0], max = 2*etalon_mirror[0])
        params.add('freq_1_%i'%(pressure_num), 1 / etalon_mirror[1], vary = True, min = (1 / etalon_mirror[1])*0.8, max = (1 / etalon_mirror[1])*1.2)
        params.add('phase_1_%i'%(pressure_num),0, vary = True, min = -np.pi, max = np.pi)
        #params.add('alpha_1_%i'%(pressure_num), 1, vary = False, min = 0.95, max = 1.05)
    if etalon_2_on == False:  
        params.add('amp_2_%i'%(pressure_num),0, vary=False)
        #params.add('freq_2_%i'%(pressure_num),1 / etalon_window[1], vary = False)
        params.add('phase_2_%i'%(pressure_num),0, vary =False)
        #params.add('alpha_2_%i'%(pressure_num), 1, vary = False)
    if etalon_2_on == True:
        params.add('amp_2_%i'%(pressure_num),etalon_window[0], vary=True)
        #params.add('freq_2_%i'%(pressure_num),1 / etalon_window[1], vary = False, min = (1 /  etalon_window[1])*0.8, max = (1 / etalon_window[1])*1.2)
        params.add('phase_2_%i'%(pressure_num),0, vary =True, min = -np.pi, max = np.pi)
        #params.add('alpha_2_%i'%(pressure_num), 1, vary = False, min = 0.98, max = 1.02)



# Define Lineshape parameters
if DBfromsource == True:
    Cond = ('AND',('BETWEEN','nu',np.floor(DATA[x_column].min() - 4*wave_range),np.ceil(DATA[x_column].max() + 4*wave_range)),('>=','sw',IntensityThreshold))
    #select('Drouin_2016_paper',Conditions=Cond,DestinationTableName='tmp')
    select('HITRAN_O2',Conditions=Cond,DestinationTableName='tmp')
if fix_temp_dep == True:
    LOCAL_TABLE_CACHE['tmp']['data']['n_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['n_air'])*[1]
    LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'])*[0]
    #LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'])*[1] #nuVC
    #LOCAL_TABLE_CACHE['tmp']['data']['n_He'] = len( LOCAL_TABLE_CACHE['tmp']['data']['n_He'])*[0] #aw temp
elif fix_temp_dep == False:
    # make so it sets to correct values
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['n_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val] = 1
        if LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][val] = 0
        #if LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][val] == 0:
            #LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][val] = 1
        #if LOCAL_TABLE_CACHE['tmp']['data']['n_He'][val] == 0:
            #LOCAL_TABLE_CACHE['tmp']['data']['n_He'][val] = 0        

if lineshape == 'VP': #lineshape = 'VP' #VP, NGP, NGP-LM, SDVP-LM, SDNGP, SDNGP-LM
    sd_vary = False
    nuVC_vary = False
    y_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['SD_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])*[0]
    LOCAL_TABLE_CACHE['tmp']['data']['y_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['y_air'])*[0]
    LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])*[0]

elif lineshape =='NGP':
    sd_vary = False
    y_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['SD_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])*[0]
    LOCAL_TABLE_CACHE['tmp']['data']['y_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['y_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] = (diffusion_coefficient(296, 0.178)) 

elif lineshape =='NGP-LM':
    sd_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['SD_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] = (diffusion_coefficient(296, 0.178)) 
elif lineshape =='SDVP':
    nuVC_vary = False
    y_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['y_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['y_air'])*[0]
    LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] == 0:
            m_a = (molecularMass(LOCAL_TABLE_CACHE['tmp']['data']['molec_id'][val],LOCAL_TABLE_CACHE['tmp']['data']['local_iso_id'][val]))
            m_p = 28.81032     
            m_rat = (m_p / m_a) / (1 + (m_p / m_a))
            a_w_ = (1- LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val])*(2/3)*m_rat
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] = a_w_#*LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][val]

elif lineshape =='SDVP-LM':
    nuVC_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] == 0:
            m_a = (molecularMass(LOCAL_TABLE_CACHE['tmp']['data']['molec_id'][val],LOCAL_TABLE_CACHE['tmp']['data']['local_iso_id'][val]))
            m_p = 28.81032     
            m_rat = (m_p / m_a) / (1 + (m_p / m_a))
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] = (1- LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val])*(2/3)*m_rat#*LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][val]
elif lineshape =='SDNGP':
    y_vary = False
    LOCAL_TABLE_CACHE['tmp']['data']['y_air'] = len( LOCAL_TABLE_CACHE['tmp']['data']['y_air'])*[0]
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] == 0:
            m_a = (molecularMass(LOCAL_TABLE_CACHE['tmp']['data']['molec_id'][val],LOCAL_TABLE_CACHE['tmp']['data']['local_iso_id'][val]))
            m_p = 28.81032     
            m_rat = (m_p / m_a) / (1 + (m_p / m_a))
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] = (1- LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val])*(2/3)*m_rat#*LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][val]
        if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] =(diffusion_coefficient(296, 0.178))
elif lineshape =='SDNGP-LM':
    for val in range(0, len( LOCAL_TABLE_CACHE['tmp']['data']['SD_air'])):
        if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] == 0:
            m_a = (molecularMass(LOCAL_TABLE_CACHE['tmp']['data']['molec_id'][val],LOCAL_TABLE_CACHE['tmp']['data']['local_iso_id'][val]))
            m_p = 28.81032     
            m_rat = (m_p / m_a) / (1 + (m_p / m_a))
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][val] = (1- LOCAL_TABLE_CACHE['tmp']['data']['n_air'][val])*(2/3)*m_rat#*LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][val]
        if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] == 0:
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][val] = (diffusion_coefficient(296, 0.178))


for i in range(0, length('tmp')):
    if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DATA[x_column].min() - wave_range) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max() + wave_range):
        if LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] >= Fit_Intensity:
            params.add('nu_%i'%(i),np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu'][i]), vary = center_vary, min = np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu'][i])-0.25,  max = np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu'][i])+0.25)
            params.add('sw_%i'%(i),np.float(LOCAL_TABLE_CACHE['tmp']['data']['sw'][i]), vary =False)
            params.add('gamma_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i]), vary = width_vary, min = 0.5*np.float(LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i]), max =  2*np.float(LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i]))
            params.add('n_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['n_air'][i]), vary = width_temp_vary, min = 0, max = 2)
            params.add('delta_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i]), vary = shift_vary, min = 0.1*np.float(LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i]), max =  10*np.float(LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i]))
            params.add('deltap_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][i]), vary = shift_temp_vary, min = -1, max = 1)           


            if LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i] != 0:
                params.add('SD_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), vary = sd_vary, min = 0.1* np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), max = 10*np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]))#, min = 0.5*np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), max =  2*np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]))
                #params.add('n_He_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i]), vary = sd_temp_vary, min = -2, max = 2)
            else:
                params.add('SD_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), vary = False)
                #params.add('n_He_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i]), vary = False)

            if LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i] != 0:
                params.add('y_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i]), vary = y_vary, min = -0.1, max = 0.1)#, min = -10*np.abs(np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i])), max =  10*np.abs(np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i])))
            else:
                params.add('y_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i]), vary = y_vary)

            if LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i] != 0:
                params.add('nu_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i]), vary = nuVC_vary, min = 0.1*np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i]), max = 10*np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i]) ) 
                #params.add('kappa_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i]), vary = nuVC_temp_vary, min = -2, max = 2) 
            else:
                params.add('nu_HT_air_%i'%(i), 0, vary = False)
                #params.add('kappa_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i]), vary = False)

            if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i]> DATA[x_column].min()) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max()):
                for pressure_num in pressure_list:
                    params.add('y_factor_%i_%i'%(i, pressure_num),1, vary = intensity_vary, min = 0.5, max = 2) 


            if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].min()) or (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DATA[x_column].max()):
                params.add('sw_%i'%(i),np.float(LOCAL_TABLE_CACHE['tmp']['data']['sw'][i]), vary = intensity_vary)

                params.add('nu_%i'%(i),np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu'][i]), vary = False)

                params.add('delta_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i]), vary = False)
                params.add('deltap_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][i]), vary = False)

                params.add('SD_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i]), vary = False)
                #params.add('n_He_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i]), vary = False)

                params.add('nu_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i]), vary = False)
                #params.add('kappa_HT_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i]), vary = False)

                params.add('y_air_%i'%(i), np.float(LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i]), vary = False)

if DBfromsource == False:
    for i in range(0, length('tmp')):
        if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DATA[x_column].min() - wave_range) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max() + wave_range):
            if LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] >= Fit_Intensity:
                if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i]> DATA[x_column].min()) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max()):
                    for pressure_num in pressure_list:
                        print (pressure_num)
                        params.add('y_factor_%i_%i'%(i, pressure_num),np.float(str(result.params['y_factor_%i_%i'%(i, pressure_num)]).split()[2][6:]) , vary = intensity_vary, min = 0.5, max = 2) 



if copy_etalon == True:
    for pressure_num in pressure_list:

        params.add('y_a_%i'%(pressure_num), np.float(str(result.params['y_a_%i'%(pressure_num)]).split()[2][6:]), vary = baseline_vary)
        if baseline_order >0:
            params.add('y_b_%i'%(pressure_num), np.float(str(result.params['y_b_%i'%(pressure_num)]).split()[2][6:]), vary = baseline_vary, min = -1, max = 1)
        if baseline_order > 1:
            params.add('y_c_%i'%(pressure_num), np.float(str(result.params['y_c_%i'%(pressure_num)]).split()[2][6:]), vary = baseline_vary, min = -1, max = 1)

        if etalon_1_on == True:
            if np.float(str(result.params['amp_1_%i'%(pressure_num)]).split()[2][6:]) != 0:
                params.add('amp_1_%i'%(pressure_num),np.float(str(result.params['amp_1_%i'%(pressure_num)]).split()[2][6:]), vary=True, min = 0.5*etalon_mirror[0], max = 2*etalon_mirror[0])
                params.add('phase_1_%i'%(pressure_num),np.float(str(result.params['phase_1_%i'%(pressure_num)]).split()[2][6:]), vary = True, min = -np.pi, max = np.pi)
                #params.add('alpha_1_%i'%(pressure_num),np.float(str(result.params['alpha_1_%i'%(pressure_num)]).split()[2][6:]), vary = False, min = 0.98, max =1.02)
        if etalon_2_on == True:
            if np.float(str(result.params['amp_2_%i'%(pressure_num)]).split()[2][6:]) != 0:
                params.add('amp_2_%i'%(pressure_num),np.float(str(result.params['amp_2_%i'%(pressure_num)]).split()[2][6:]), vary=True, min = etalon_window[0]*0.5, max = etalon_window[0]*2)


for pressure_num in pressure_list:
    if lock_baseline == True:
        params.add('y_a_%i'%(pressure_num), np.float(str(result.params['y_a_%i'%(pressure_num)]).split()[2][6:]), vary =False)
        params.add('y_b_%i'%(pressure_num), np.float(str(result.params['y_b_%i'%(pressure_num)]).split()[2][6:]), vary =False)
        params.add('y_c_%i'%(pressure_num), np.float(str(result.params['y_c_%i'%(pressure_num)]).split()[2][6:]), vary =False)

    if lock_etalons == True:

        if etalon_1_on == True:
            params.add('amp_1%i'%(pressure_num),np.float(str(result.params['amp_1%i'%(pressure_num)]).split()[2][6:]), vary=False)
            params.add('phase_1%i'%(pressure_num),np.float(str(result.params['phase_1%i'%(pressure_num)]).split()[2][6:]), vary = False)
            #params.add('alpha_1%i'%(pressure_num),np.float(str(result.params['alpha_1%i'%(pressure_num)]).split()[2][6:]), vary = False)
        if etalon_2_on == True:
            params.add('amp_2%i'%(pressure_num),np.float(str(result.params['amp_2%i'%(pressure_num)]).split()[2][6:]), vary=False)
            params.add('phase_2%i'%(pressure_num),np.float(str(result.params['phase_2%i'%(pressure_num)]).split()[2][6:]), vary =False)
            #params.add('alpha_2%i'%(pressure_num),np.float(str(result.params['alpha_2%i'%(pressure_num)]).split()[2][6:]), vary = False)
        if etalon_3_on == True: 
            params.add('amp_3%i'%(pressure_num),np.float(str(result.params['amp_3%i'%(pressure_num)]).split()[2][6:]), vary=False)
            params.add('phase_3%i'%(pressure_num),np.float(str(result.params['phase_3%i'%(pressure_num)]).split()[2][6:]), vary =False)
            #params.add('alpha_3%i'%(pressure_num),np.float(str(result.params['alpha_3%i'%(pressure_num)]).split()[2][6:]), vary = False)
        if etalon_4_on == True:
            params.add('amp_4%i'%(pressure_num),np.float(str(result.params['amp_4%i'%(pressure_num)]).split()[2][6:]), vary=False)
            params.add('phase_4%i'%(pressure_num),np.float(str(result.params['phase_4%i'%(pressure_num)]).split()[2][6:]), vary =False)
            #params.add('alpha_4%i'%(pressure_num),np.float(str(result.params['alpha_4%i'%(pressure_num)]).split()[2][6:]), vary = False)


ignore_pts = []
filter_pts = []


for pt in ignore_pts:
    DATA.loc[(DATA.index == pt) , 'Filter'] = 1
for pt in filter_pts:
    DATA.loc[(DATA.index == pt)  , 'Filter'] = 2

x = DATA[(DATA['Filter'] ==0) ][x_column].values
x_min =  DATA[(DATA['Filter'] ==0) ][x_column].values.min()
y = DATA[(DATA['Filter'] ==0) ][fit_column].values
pressure_label =   100#DATA[(DATA['Filter'] ==0) ]['Nominal Pressure']  



minner = Minimizer(sim_model, params,fcn_args=(x, y, pressure_list, DATA, x_min, fit_column, x_column), iter_cb = max_iter)
result = minner.minimize(method = 'leastsq')#'
final = y + result.residual

fit_results = pd.DataFrame()
fit_results['Pressure'] = pressure_label
fit_results['Wavenumber'] = x
fit_results['Experiment'] = y
fit_results['Model'] = final
fit_results['Residuals'] = result.residual

result.params.pretty_print()


#bkg_result, param_result = summary_result_file(result, lineshape)
#print (param_result)
#print (bkg_result)


'''
# Update Lineshape parameters
for i in range(0, length('tmp')):
    if (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] > DATA[x_column].min() - wave_range) and (LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] < DATA[x_column].max() + wave_range):
        if LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] >= Fit_Intensity:
            LOCAL_TABLE_CACHE['tmp']['data']['nu'][i] = np.float(str(result.params['nu_%i'%(i)]).split()[2][6:]) 
            LOCAL_TABLE_CACHE['tmp']['data']['sw'][i] = np.float(str(result.params['sw_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['gamma_air'][i] = np.float(str(result.params['gamma_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['delta_air'][i] = np.float(str(result.params['delta_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['SD_air'][i] = np.float(str(result.params['SD_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['y_air'][i] = np.float(str(result.params['y_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['nu_HT_air'][i] = np.float(str(result.params['nu_HT_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['n_air'][i] = np.float(str(result.params['n_air_%i'%(i)]).split()[2][6:])
            LOCAL_TABLE_CACHE['tmp']['data']['deltap_air'][i] = np.float(str(result.params['deltap_air_%i'%(i)]).split()[2][6:])
            #LOCAL_TABLE_CACHE['tmp']['data']['n_He'][i] = np.float(str(result.params['n_He_%i'%(i)]).split()[2][6:])
            #LOCAL_TABLE_CACHE['tmp']['data']['kappa_HT_air'][i] = np.float(str(result.params['kappa_HT_air_%i'%(i)]).split()[2][6:])
'''  
fig = plt.figure(figsize = (16,10))
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])

QF = np.around((fit_results['Experiment'].max() - fit_results['Experiment'].min())  / fit_results['Residuals'].std(), 0)
ax0 = plt.subplot(gs[0])


ax0.plot(fit_results['Wavenumber'],fit_results['Model'] )
ax0.plot(fit_results['Wavenumber'], fit_results['Experiment'])
ax0.text(0.25, 0.95,'QF: ' + str(QF), horizontalalignment='center', verticalalignment='center', transform = ax0.transAxes)

ax1 = plt.subplot(gs[1])
ax1.plot(fit_results['Wavenumber'],fit_results['Residuals'], "-")
plt.show()



#print (sample , tau_correction, result.params['y_factor_11_100'].value, result.params['y_factor_15_100'].value, QF)

os.chdir(path)


#Saves fit results as a Dataframe
fit_results.to_csv(save_name + '_spectra.csv', index = False)
#fit_param_result = param_result.append(bkg_result)
#fit_param_result.to_csv(save_name + '_params.csv', index = False)







Exception: HITRAN_O2: no such table. Check tableList() for more info.